# 6 - Ranking and Recommender Models
## CSCI E-108
### Steve Elston

Recommender algorithms are a vast and complex subject. A comprehensive treatment is impractical in a reasonable time frame. Rather, the exercises in this notebook are formulated to increase and evaluate your understanding of the basic principles of recommender algorithms. 

> **Note:** The code provided in this notebook was created and tested in Google Colab. With a few file-system specific modification this notebook should execute in your local environment if you so choose.  

We do not use any particular recommender package for these exercises. To get started, execute the code in the cell below to import the required packages.     

In [ ]:
import os
import requests
import zipfile
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, vstack
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

%matplotlib inline

> **Attribution:** Much of the code in this notebook has been modified from machine generated code from Microsoft Copilot and Gemini.   

## MovieLens Dataset    

For the remainder of this notebook you will work the [MovieLens 100k dataset](https://grouplens.org/datasets/movielens/100k/). The dataset contains 100,000 ratings of movies by a number of users.    

Execute the code in the cell below to download and unzip the dataset.        

In [ ]:
# URL and target paths
url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
zip_path = "ml-100k.zip"
extract_dir = "ml-100k"

# Download the dataset
print("Downloading MovieLens 100k dataset...")
response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)

# Extract the zip file
print("Extracting dataset...")
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall()

# Clean up zip file
os.remove(zip_path)
print(f"\nDataset extracted to directior: {extract_dir}  With files:")
!ls ml-100k

Next, execute the code in the cell below to map the zip files to a Pandas dataframe.   

In [ ]:
path = extract_dir + 'u1.base'
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
ratings['datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')
print(f"Number of ratings: {len(ratings)}")
ratings.head()

There are three key columns in this data frame, the user ID, the item ID, and the rating. There are a total 100,000 ratings in the data frame.     
Next, execute the code in the cell below to instantiate a data frame containing the movie details.

In [ ]:
cols = [
    'movie id',
    'movie title',
    'release date',
    'video release date',
    'IMDb URL',
    'unknown',
    'Action',
    'Adventure',
    'Animation',
    'Children\'s',
    'Comedy',
    'Crime',
    'Documentary',
    'Drama',
    'Fantasy',
    'Film-Noir',
    'Horror',
    'Musical',
    'Mystery',
    'Romance',
    'Sci-Fi',
    'Thriller',
    'War',
    'Western'
]

movies = pd.read_csv('ml-100k/u.item',  sep='|', encoding='latin-1', header=None)
movies.columns = cols
movies.head()

## Explore the Dataset    

We will now explore the ratings data. To start, execute the code in the cell below to find the number of unique values of the variables in the ratings data frame.  

In [ ]:
unique_vals = ratings.nunique()
print(f"Number of unique values in dataset\n {unique_vals}")

There are only a limited number of users and movies in the this dataset.    

To explore the changes in the number of ratings with time, execute the code in the cell below.      

In [ ]:
g = sns.jointplot(data=ratings, x='timestamp', y='rating', kind='hex', height=6, ratio=4);
sns.regplot(data=ratings,
            x='timestamp',
            y='rating',
            ax=g.ax_joint,
            scatter=False,
            color='red',
            line_kws={'linestyle': '--', "linewidth": 1},
            ci=95);
plt.suptitle("Ratings vs. time\n Regression line in red", y=1.02);

To get a feeling for the distribution of ratings done by the users and the number of ratings for the items run the code in the cell below.

In [ ]:
rating_counts = ratings.loc[:,"user_id"].value_counts()
item_counts = ratings.loc[:,"user_id"].value_counts()

_, ax = plt.subplots(1,2, figsize=(10,4));
ax[0].hist(rating_counts, bins=100);
ax[0].set_title('Histogram of number of ratings for users');
ax[0].set_xlabel('Number of ratings');
ax[0].set_ylabel('Count');

ax[1].hist(item_counts, bins=100);
ax[1].set_title('Histogram of number of ratings for items');
ax[1].set_xlabel('Number of ratings');
ax[1].set_ylabel('Count');

The number of ratings by both users and items shows an exponential or power-law distribution. Most users and items have very few ratings. However, some items and ratings have a great many ratings.   

## Creating Sparse Train and Test Matrices  

The **utility matrix** has high dimensions and is extremely sparse. This fact can be understood by considering the small number of reviews compared to the dimensions of the matrix. In this case, we have the users in the rows and the items in the columns. Given the high dimensionality and sparsity of the utility matrix representing this data structure as a full dense matrix with missing values is undesireable. In fact, for massive scale recommenders there is no reasonable cluster with sufficient memory accommodate such a data structure. Fortunately, there is an alternative, use a **hash table**. The non-missing entries of the utility matrix are then stored as key-value pairs. The key for the sparse matrix data structure is the row-column tuple, which is hashed for reference to the value.     

The code in the cell below has two major steps:    
1. The data frame containing the user reviews is converted to a [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) matrix.
2. The sparse matrix is sampled into train and test subsets. The sampling is done randomly by rating. We do not want to simply sample by users or items as we need to be able to train and test on ratings.

Execute this code and examine the results.  

In [ ]:
def create_spare_matrix(matrix):
  """
  Function to create a sparse csr_matrix from a dataframe of user ratings

  Args: matrix - a data frame with the user-item ratings in the rows

  Returns: a sparse csr_matrix
  """
  # Adjust user and item IDs to be zero-based (important for sparse matrix indexing)
  matrix['user_id'] -= 1
  matrix['item_id'] -= 1

  # Create sparse matrix: rows = users, columns = items, values = ratings
  num_users = matrix['user_id'].max() + 1
  num_items = matrix['item_id'].max() + 1

  sparse = csr_matrix(
    (matrix['rating'], (matrix['user_id'], matrix['item_id'])),
    shape=(num_users, num_items)
  )
  return sparse

def train_test_split_sparse(matrix, test_fraction=0.2, random_state=None):
    # Create a sparse matrix
    sparse = create_spare_matrix(matrix)

    # Convert to COO format for easy indexing
    sparse_coo = sparse.tocoo()
    rng = np.random.default_rng(random_state)

    # Get all non-zero indices
    n_samples = sparse_coo.nnz
    indices = np.arange(n_samples)
    test_size = int(n_samples * test_fraction)

    # Randomly select test indices
    test_idx = rng.choice(indices, size=test_size, replace=False)
    train_idx = np.setdiff1d(indices, test_idx)

    # Build train and test sparse matrices
    train = csr_matrix((sparse_coo.data[train_idx],
                        (sparse_coo.row[train_idx], sparse_coo.col[train_idx])),
                       shape=sparse_coo.shape)

    test = csr_matrix((sparse_coo.data[test_idx],
                       (sparse_coo.row[test_idx], sparse_coo.col[test_idx])),
                      shape=sparse_coo.shape)

    return train, test

sparse_ratings_train, sparse_ratings_test = train_test_split_sparse(ratings, test_fraction=0.2, random_state=543)

print(f"Number of training users: {sparse_ratings_train.shape[0]}")
print(f"Number of training items: {sparse_ratings_train.shape[1]}")
print(f"Number of non-zero entries in sparse train matrix: {sparse_ratings_train.nnz}")
print(f"\nNumber of test users: {sparse_ratings_test.shape[0]}")
print(f"Number of test items: {sparse_ratings_test.shape[1]}")
print(f"Number of non-zero entries in sparse test matrix: {sparse_ratings_test.nnz}")


> **Exercise 6-01:** The question one should think about is how much memory does the sparse matrix (hash table) representation save? We can quantify this by the compression ratio:
> $$compression\ ratio = \frac{size\ of\ sparse\ matrix}{size\ of\ dense\ matrix}$$
> You will now work this out for two cases. Assume that each number requires 8 bytes of memory for the sparse representation. Keep in mind that the key is actually a two value tuple, $(user_ID, item_ID$. We can ignore the small overhead of the row and column indices for the dense representation. Assume all 100,000 ratings are in one matrix.    
> 1. In the cell below create code to compute a) the size of the full matrix, b) the size of the sparse matrix, and c) the compression achieved using the sparse representation.          
> 2. Next perform the same calculations for a more realistic size systems with 1 million items and 100 million users and an average of 20 ratings per user.     

In [ ]:
## Put your code here






> **End of Exercise**

The next step is to filter the train and test datasets to remove items or users with too few ratings. Having items or users with too few ratings increases the error of rating predictions and can lead to an over-fit model. Execute the code in the cell below to apply these filters.   

In [ ]:
def filter_users(ratings_train, ratings_test, min_rating_count):
  # Get the count of nonzero values in rows
  row_counts_train = ratings_train.count_nonzero(axis=1)
  row_counts_test = ratings_test.count_nonzero(axis=1)

  # Create and apply a boolean mask for rows with enough nonzeros
  mask = (row_counts_train >= min_rating_count) & (row_counts_test >= min_rating_count)
  return ratings_train[mask, :], ratings_test[mask,:]

def filter_items(ratings_train, ratings_test, min_rating_count):
  # Get the count of nonzero values in rows
  col_counts_train = ratings_train.count_nonzero(axis=0)
  col_counts_test = ratings_test.count_nonzero(axis=0)

  # Create and apply a boolean mask for rows with enough nonzeros
  mask = (col_counts_train >= min_rating_count) & (col_counts_test >= min_rating_count)
  return ratings_train[:, mask], ratings_test[:, mask]


min_rating_count = 5
sparse_ratings_train, sparse_ratings_test = filter_users(sparse_ratings_train, sparse_ratings_test, min_rating_count)
sparse_ratings_train, sparse_ratings_test = filter_items(sparse_ratings_train, sparse_ratings_test, min_rating_count)

print(f"Number of training users: {sparse_ratings_train.shape[0]}")
print(f"Number of training items: {sparse_ratings_train.shape[1]}")
print(f"Number of non-zero entries in sparse train matrix: {sparse_ratings_train.nnz}")
print(f"\nNumber of test users: {sparse_ratings_test.shape[0]}")
print(f"Number of test items: {sparse_ratings_test.shape[1]}")
print(f"Number of non-zero entries in sparse test matrix: {sparse_ratings_test.nnz}")

> **Exercise 6-02:** Consider the effects that these filtering operations have on the results expected from any recommender system and answer these questions.
> 1. How will this filtering impact the ability of the recommender to show items in the 'long tail'?
> 2. How will this filtering impact the user cold start problem for the recommender?  

> **Answers:**
> 1.          
> 2.     

## Create a Baseline Model

A baseline recommender model imputes the mean rating for each item. The mean implied rating is the sum of the mean rating for a user and the mean rating for the item. For user $x$ and item $i$ the implied baseline rating is:  
$$\hat{r}_{x,i} = \mu + \bar{r}_x + \bar{r}_i$$     
where:  
$\mu$ is the overall mean rating for all users and items.     
$\bar{r}_x$ is the mean rating of items by user $x$.    
$\bar{r}_i$ is the mean rating of item, $i$, for all users rating that item.     

The ranks of these mean or expected implied ratings are the recommendations. In simple terms the baseline model uses 'crowd sourced' mean ratings.    

### Compute the user mean ratings

The first step of computing a baseline is to zero-mean the overall ratings along with the ratings of each user and item. This step removes biases from users with exceptionally high or low average ratings. Execute the code in the cell below to perform these operations.     

In [ ]:
def indices_to_rows(sparse):
    """Returns row index for each data entry in CSR matrix."""
    row_indices = np.empty_like(sparse.data, dtype=int)
    for i in range(len(sparse.indptr) - 1):
        row_indices[sparse.indptr[i]:sparse.indptr[i+1]] = i
    return row_indices

def compute_row_means(X):
    row_sums = X.sum(axis=1).A1  # Convert to 1D array
    row_counts = np.diff(X.indptr)
    row_counts[row_counts < 1] = 1
    return row_sums / row_counts

def compute_column_means(X):
    col_sums = X.sum(axis=0).A1
    col_counts = np.diff(X.tocsc().indptr)
    col_counts[col_counts < 1] = 1
    return col_sums / col_counts



def zero_center_sparse(X):
    """
    Computes and returns the Zero-centers sparse matrix by rows and columns,
    preserving sparsity along with the means of the rows and columns.

    Args:
        sparse_matrix (scipy.sparse.csr_matrix): The input sparse matrix.
        axis: the axis along which to center the values

    Returns:
        scipy.sparse.csr_matrix: The zero-centered sparse matrix.
        Mean: the overall mean rating for all items and users.
        User_means: a vector of the user or row means
        item_means: a vector of the item or column means
    """
    if not isinstance(X, csr_matrix):
        X = X.tocsr()

    # Subtract overall mean from non-zero entries
    sum_ratings = X.sum()
    mean_rating = sum_ratings / X.nnz
    X_new = X.copy().astype(np.float32)
    X_new.data -= mean_rating

    # Compute row means
    row_means = compute_row_means(X_new)

    # Compute column means
    col_means = compute_column_means(X_new)

    # Subtract row means from non-zero entries
    X_new.data -= row_means[indices_to_rows(X_new)]


    # Subtract column means from non-zero entries
    X_new.data -= col_means[X_new.indices]

    return X_new, mean_rating, row_means, col_means

centered_spare_ratings_train, mean_rating, user_means, item_means = zero_center_sparse(sparse_ratings_train)

With the means and centered sparse utility matrix computed, let's explore the result. Execute the code in the cell below to display the mean rating along with histograms of the zero-centered rating, the centered user ratings and the centered item ratings.  

In [ ]:
print(f"Mean rating: {mean_rating}")

plt.figure(figsize=(8, 4))
plt.hist(centered_spare_ratings_train.data, bins=50);
plt.title('Distribution zero-centered ratings')
plt.xlabel('rating');
plt.ylabel('Count');
plt.show()

plt.figure(figsize=(8, 4))
plt.hist(user_means, bins=50);
plt.title('Distribution of user means')
plt.xlabel('rating');
plt.ylabel('Count');
plt.show()

plt.figure(figsize=(8, 4))
plt.hist(item_means, bins=50);
plt.title('Distribution of item means')
plt.xlabel('rating');
plt.ylabel('Count');
plt.show()

> **Exercise 6-03:** Examine the foregoing histograms and answer these questions.
> 1. Notice that the bulk of the zero-centered ratings are in the range of about $(-2,2)$. What does this range of variation tell you about the overall ratings?
> 2. Notice that the bulk of the user mean ratings are in a narrow range of about $(-0.8,0.8)$ and the mean item ratings are in a narrow range of about $(-1.3,0.7)$. What do these results tell you about the variation in ratings from item to item and user to user?    

> **Answers:**   
> 1.         
> 2.   

### Compute the Baseline Rating Predictions   

With the mean ratings for items and users computed it is time to compute the baseline rating predictions. Conceptually, this is a simple matter of adding the means for each user_ID, item_ID tuple. Execute the code in the cell below to do just this and return the baseline implied ratings.   

In [ ]:
def baseline_rating_predict(rating_mean, means_user, means_item, sparse_ratings_test):
  # Create a new sparse matrix with zero values
  shape = sparse_ratings_test.shape

  # Get the indptr and indices arrays from the template matrix
  indptr = sparse_ratings_test.indptr
  indices = sparse_ratings_test.indices

  # Create a new empty sparse matrix
  data = csr_matrix((shape), dtype=sparse_ratings_test.dtype)

  ## Find the row, column and index
  rows, cols = sparse_ratings_test.nonzero()

  ## Add the row and column means
  means_temp_user = means_user + rating_mean
  data = means_item[cols] + means_temp_user[rows]

  return csr_matrix((data, indices, indptr), shape=shape)

predicted_baseline_ratings = baseline_rating_predict(mean_rating, user_means, item_means, sparse_ratings_test)
predicted_baseline_ratings

### Evaluate the Baseline Predicted Ratings  

With the baseline implied ratings computed we need to evaluate the results. The code in the cell below does the following:     
1. The difference between the actual rating and the implied rating is computed.
2. Some basic error statistics are computed.
3. A plot of the errors vs. the actual rating is displayed.

Execute the code and examine the results.  

In [ ]:
def evaluate_recommender(ratings_test, predicted_ratings):

  ## Compute the sparse error array
  ratings_errors = predicted_ratings - ratings_test

  ## Create data frame from the sparse test array
  ratings_test_coo = ratings_test.tocoo()
  rating_test_df = df = pd.DataFrame({
    'row_index': ratings_test_coo.row,
    'col_index': ratings_test_coo.col,
    'Rating': ratings_test_coo.data
  })

  ## Create data frame from the sparse error array
  ratings_errors_coo = ratings_errors.tocoo()
  rating_errors_df = df = pd.DataFrame({
    'row_index': ratings_errors_coo.row,
    'col_index': ratings_errors_coo.col,
    'Rating_Error': ratings_errors_coo.data
  })
  rating_errors_df.loc[:,'Rating'] = rating_test_df.loc[:,'Rating']

  ## Print some error statistics
  print(f"Mean error: {round(rating_errors_df.loc[:,'Rating_Error'].mean(),3)}")
  print(f"Root mean square error: {round(rating_errors_df.loc[:,'Rating_Error'].std(),3)}")
  print(f"Median absolute error: {round(rating_errors_df.loc[:,'Rating_Error'].abs().median(),3)}\n")

  ## PLot the error distributions
  g = sns.jointplot(data=rating_errors_df, x='Rating', y='Rating_Error',
                  kind='hex', height=6, ratio=4);
  sns.regplot(data=rating_errors_df,
            x='Rating',
            y='Rating_Error',
            ax=g.ax_joint,
            scatter=False,
            color='red',
            line_kws={'linestyle': '--', "linewidth": 1},
            ci=95);
  plt.suptitle("Rating error vs. rating\n Regression line in red", y=1.02);

evaluate_recommender(sparse_ratings_test, predicted_baseline_ratings)

> **Exercise 6-04:** Examine the plot of the error vs the actual rating. Note the systematic bias that changes with actual rating. Given the baseline model, how can you explain this effect?  

> **Answer:**    

## K Nearest Neighbor Search Collaborative Filtering  

**Collaborative filtering** algorithms search for the highest similarity between behavior or engagement between items or between users.
For this algorithm we will apply a simple **k-nearest-neighbor search** algorithm. In this case we will do user-user nearest neighbor search. The search is performed between a user's recommendation profile, the query, $q$, and the recommendation profiles of the other users, $u$. The $k$ nearest neighbor ratings to the query are used to compute the implied ratings. The similarities, $s(r_q, r_u)$, are used as weights to compute the mean rating, $\hat{r}_i$, for each item, $i$.
$$\hat{r}_i = \frac{\sum^k_{u=1} s(r_{q,i}, r_{u,i}) \times r_{u,i}}{\sum^k_{u=1} s(r_{q,i}, r_{u,i})}$$

The major steps in this algorithm are:      
1. The query rating profile vector and profile vectors of the other users are zero-centered and variance normalized. This allows **cosine similarity** to be computed using dot products.
2. The cosine similarity between the query and the other user's normalized vectors is computed.
3. The similarities are ranked, or sorted in descending order, and the top $k$ are retained.
4. The implied rating is computed as the weighted average of the ratings of the top $k$ users.
5. The implied ratings for the query user are sorted in descending order and the top few are retained.
6. The items with the highest implied ratings are the recommendations.
 

> **Exercise 6-05:** In one or a few sentences explain the key difference between the learning in content-based recommender algorithms and collaborative filtering recommender algorithms.

> **Answer:**       


The code in the cell below computes the $k$ nearest neighbor users to the query user, returning the neighbor user ids and the similarities. Execute this code and examine the results.  

In [ ]:
def normalize_ratings(ratings):
    """
    Function returns the normalized user ratings (rows) of the
    sparse utility matrix.
    """
    n_rows, n_cols = ratings.shape
    X = ratings.copy()  # avoid modifying original

    new_data = []
    new_indices = []
    new_indptr = [0]

    for i in range(n_rows):
        row_start, row_end = X.indptr[i], X.indptr[i + 1]
        indices = X.indices[row_start:row_end]
        data = X.data[row_start:row_end]

        nnz = len(data)
        mean = data.sum() / nnz
        norm = np.sqrt(np.sum((data - mean) ** 2))
        #var = sq_sum / nnz

        new_data.extend((data - mean) / norm)

        new_indices.extend(indices)
        new_indptr.append(len(new_data))

    return csr_matrix((new_data, new_indices, new_indptr), shape=X.shape)


def normalize_query(query):
    """
    Function to compute zero mean unit variance query vector
    """
    nnz = query.nnz
    data = query.data

    # zero mean the data
    mean = data.sum() / nnz
    data = data - mean

    # Variance
    #var = np.sum((data) ** 2) / nnz
    norm = np.sqrt(np.sum((data) ** 2))

    # normalize by standard deviaton
    data = data / norm
    return csr_matrix((data, query.indices, query.indptr), shape=query.shape)


def user_cosine_similarity(ratings, query, k=20):
  # Variance normalize the vectors
  ratings_normalized = normalize_ratings(ratings)
  query_normalized = normalize_query(query)

  rating_similarity = ratings_normalized.dot(query_normalized.T).toarray().flatten()
  out = pd.DataFrame({'user':range(ratings.shape[0]), 'similarity':rating_similarity})
  return out.sort_values('similarity', ascending=False)[:k]

similarity_query = user_cosine_similarity(centered_spare_ratings_train, sparse_ratings_test[0,:], k=50)
similarity_query

With the similarities for the top $k$ nearest neighbors computed, the next step is to compute the implied ratings as the similarity weighted average of the ratings. One needs to be careful to only include the non-zero elements of each column (item) vector when making this calculation. Execute the code in the cell below to compute the implied ratings for all items.   

In [ ]:
def knn_implied_ratings(ratings, similarities, item_ratings):
  mask = [True if i in similarities.user else False for i in range(ratings.shape[0])]
  X = ratings[mask,:].copy()
  weighted_rating = np.array([0.0]*X.shape[1])

  for j in range(X.shape[1]):
    col = X[:, j]
    rows = col.nonzero()[0]
    row_mask = [True if k in rows else False for k in range(X.shape[0])]
    col_weight = np.sum(similarities.loc[row_mask, 'similarity'])
    if col_weight > 0:
        weighted_rating[j] = np.dot(col.toarray().flatten().T, similarities.loc[:, 'similarity'])/col_weight
    else:
        weighted_rating[j] = item_ratings[j]
  return  weighted_rating

implied_ratings = knn_implied_ratings(centered_spare_ratings_train, similarity_query, item_means)

As a check on the implied rating calculation execute the code in the cell below to display a histogram. Keep in mind that we are still working with zero-centered rating values.  

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(implied_ratings, bins=30);
plt.title('Distribution zero-centered ratings')
plt.xlabel('rating');
plt.ylabel('Count');
plt.show()

Finally, we are ready to compute the recommendations. To insure that recommendations are generated only for items the query user has not already rated, a mask is created. The implied ratings are sorted in descending order and the list truncated, which is the recommendation list. Execute the code in the cell below and examine the result.   

In [ ]:
def knn_recommender(query, ratings, k):
  rated_items = query.nonzero()[1]
  mask = [False if i in rated_items else True for i in range(query.shape[1])]
  # mask = np.where(ratings[user_id, :].toarray()[0] == 0)[0]
  unrated = ratings[mask]
  out = pd.DataFrame({'item_index':range(len(unrated)), 'ratings': unrated})
  return out.sort_values('ratings', ascending=False)[:k]


knn_recommendations = knn_recommender(sparse_ratings_test[0,:], implied_ratings, k=10)
knn_recommendations

To make this recommendation list human-understandable, it is easy to find the movie titles, using the `item_id`. Execute the code in the cell below and examine the result.  

In [ ]:
movies.loc[knn_recommendations['item_index'],'movie title']

> **Exercise 6-06:** We will not take the time to do a full evaluation of the kNN collaborative filtering algorithm. You can at this point make a few observations of the outcome for the single query and answer these questions.
> 1. One possible evaluation of the kNN collaborative filtering algorithm is to compare the distribution of the implied (estimated) ratings with the distribution of the actual zero-centered item ratings. Compare the histogram displayed above to the histogram of the zero-centered item ratings display previously. Do these two distributions appear to be approximately the same or not and what are some noticeable differences, if any? What does this tell you about the accuracy of this algorithm?
> 2. In a few sentences, explain why the simple kNN collaborative filtering algorithm used for the foregoing example is not scaleable. Try to be as specific as you can.    

> **Answers:**
> 1.      
> 2.       

## Collaborative Filtering by Matrix Decomposition    

Given the limited scalability of the basic kNN search algorithm, we need to look elsewhere. The solution is to apply **latent variable** models. These models use various techniques **dimensionality reduction** to compress the high-dimensional utility matrix space to a lower dimensional **latent space**. There are two general approaches to learning latent spaces:   
1. **Unsupervised learning** algorithms which compute a **factorization** of the utility matrix. For this example we will use [**singular value decomposition (SVD)**](https://en.wikipedia.org/wiki/Singular_value_decomposition) to learn a set of orthogonal latent variables. The orthogonalization is a linear process.        
2. **Supervised learning** algorithms compute optimal weights for a set of latent variables by reducing a loss function. There are quite a number of algorithms that have been applied to this problem, mostly of which use some form of neural networks. These algorithms are able to learn interactions between variables. We will not purse this approach further here.

### Matrix Factorization    

We can factorize the utility matrix using singular value decomposition. The implied rating matrix, $\hat{R}$, can then be computed using the factorization. This is done by matrix multiplication:     
$$\hat{R} = U \Sigma V^T$$    
Where for $X$ users and $I$ items,     
$R \in \{X\ \times I \}$ is the utility matrix containing ratings.  
$U \in \{I \times s \}$ is the matrix of left singular vectors.          
$\Sigma \in \{s \times s\}$ is the diagonal matrix of singular values with $s \le min(X, I)$.       
$V \in \{X \times s\}$ is the matrix of right singular vectors.  

By selecting a value of $s \le min(X, I)$ the SVD reduces dimensionality. The $s \times s$ diagonal matrix of singular values are the weights or **loadings** of the factors. The singular vector matrices contain the orthogonal bases for the factors.

The code in the cell below uses the [scipy.sparse.linalg.svds](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html) function to perform the SVD decomposition of the sparse utility matrix. Since this algorithm explicitly works with sparse matrices the bias from zero padding of missing values. Execute the code in the cell below to perform the SVD on the centered sparse utility matrix and displays a plot of the singular values.        

In [ ]:
k=100
U, sigma, Vt = svds(centered_spare_ratings_train, k=k)


print(f"Shape of U: {U.shape}")
print(f"Shape of V transpose: {Vt.shape} \n\n")
plt.figure(figsize=(8, 4))
x = [i+1 for i in range(len(sigma))]
plt.plot(x, sigma[::-1]);
plt.title('Singular values from utility matrix decomposition')
plt.xlabel('Index');
plt.ylabel('Singular value');
plt.show()

> **Exercise 6-07:** Notice that the graph of singular values drops rapidly at first and then at a slower steady rate. Consider the effect of increasing or decreasing $s$ the number of singular values retained.
> 1. How does the accuracy of the decomposition in representing the full matrix change as $s$ increases? How could choosing a value of S that is too large lead to an over-fit model?
> 2. Roughly how does the number of operations required to compute the implied ratings increase with $s$.
> 3. If we use 8 byte floating point numbers to represent $U$ and $V$ how big is the data structure in this cases. How does the memory required scale as we increase s?    

> **Answers:**
> 1.         
> 2.         
> 3.         

With the utility matrix factorized, we are ready to compute the implied ratings. Execute the code in the cell below to compute the implied ratings from the utility matrix decomposition.   

In [ ]:
def svd_predict_ratings(U, Vt, sigma, rating_mean, means_items, means_user):
    """
    Predicts ratings using matrix factorization, adding back user and item means.

    Args:
        U (np.ndarray): User latent factor matrix.
        V (np.ndarray): Item latent factor matrix.
        means_items (np.ndarray): Vector of item means.
        means_user (np.ndarray): Vector of user means.
        sparse_ratings_test (scipy.sparse.csr_matrix): The sparse matrix of test ratings.

    Returns:
        scipy.sparse.csr_matrix: Sparse matrix of predicted ratings.
    """
    # Compute the dot product of U and V.T
    sigma_diag = np.diag(sigma)
    sparse_predictions = csr_matrix(np.dot(np.dot(U, sigma_diag), Vt))

    ## Find the row, column and index
    rows, cols = sparse_predictions.nonzero()

    return sparse_predictions


SVD_rating_prediction = svd_predict_ratings(U, Vt, sigma, mean_rating, item_means, user_means)
evaluate_recommender(sparse_ratings_test, SVD_rating_prediction)

> **Exercise 6-08:** Compare the evaluation of the matrix factorization recommender with the baseline recommender you evaluated previously and answer these questions.
> 1. How does the accuracy and bias compare?
> 2. What do the accuracy and bias tell you about the adequacy of the SVD factorization as a recommender representation?   

> **Answers:**
> 1.       
> 2.          

There is one final step, computing the recommendations. The code in the cell below computes the implied (predicted) ratings for a query user. The ratings of unrated items are sorted in descending order and the item ids for the top rated items are returned. Execute this code and examine the results.    

In [ ]:
 def get_recommendations_for_user(predicted_ratings, ratings, user_id, n_recommendation):
    """
    Function returns item ids of recommendations given a user_id
    """
    ## Get the predicted ratings
    user_predicted_ratings = predicted_ratings[user_id, :]

    # Find unrated items for the user (assuming R_sparse has 0 for unrated)
    unrated_item_indices = np.where(ratings[user_id, :].toarray()[0] == 0)[0]

    # Get predicted ratings for unrated items
    predictions_for_unrated = user_predicted_ratings[0, unrated_item_indices]
    print(predictions_for_unrated.shape)
    recommendations = pd.DataFrame({'item_index':unrated_item_indices, 'rating': predictions_for_unrated.toarray()[0]})

    # Sort and return top recommendations
    return recommendations.sort_values(by='rating', ascending=False).iloc[:n_recommendation,:]

user = 0
n_recommendation = 10
recommendation_indicies = get_recommendations_for_user(SVD_rating_prediction,
                                                       centered_spare_ratings_train,
                                                       user, n_recommendation)

recommendation_indicies

To make this recommendation list human-understandable, it is easy to find the movie titles, using the `item_id`. Execute the code in the cell below and examine the result.  

In [ ]:
movies.loc[recommendation_indicies['item_index'],'movie title']

> **Exercise 6-09:** Notice that the recommendations computed with the kNN collaborative filtering algorithm and the matrix factorization algorithm are completely different for the same user. Beyond possible small accuracy differences between the algorithms, what does this situation tell you about the difficulty of making good recommendations from rating data?     

> **Answer:**        

## Conceptual Questions   

We will complete this assignment with a short set of conceptual questions.  

> **Exercise 6-10:** Recommender systems must overcome a number of biases. Answer the following questions concerning these biases in one or a few sentence.
> 1. Explain how negative sampling bias arises in ranking algorithms.
> 2. Explain how position bias arises in ranking algorithms.
> 3. How are negative sampling bias and position bias related?

> **Answers:**
> 1.      
> 2.        
> 3.      

> **Exercise 6-11:** Embedding is essential for similarity search required for search and recommender algorithms. Answer the following questions in one or a few sentences.
> 1. Provide an example of high-dimensional categorical variables used in recommender algorithms.   
> 2. Why is embedding of categorical features essential for recommender systems.
> 3. Explain the consequences of using a hash algorithm that does not have each of the four properties are essential for hashing a high dimensional categorical variable?
> 4. If you need to use multiple high dimensional categorical variables in a ranking algorithm how can you formulate a vector for similarity search?      

> **Answers:**
> 1.       
> 2.      
> 3.                
> 4.       

> **Exercise 6-12:** In the precious assignment you worked with models that incorporate concepts from content-based recommenders. Consider the key concepts that define content based recommenders and answer the following questions in one or a few sentance.:
> 1. What type of search is used to generate the recommendation list?
> 2. How are recommendations ranked in a content-based recommender?
> 3. How can a item-item content-based recommender overcome the cold start problem?      

> **Answers:**
> 1.       
> 2.    
> 3.         

> **Exercise 6-13:** In one or a few sentences answer the following questions about collaborative filtering algorithms.
> 1. How do mixed negative sampling (MNS) help with negative sampling bias for collaborative filtering algorithms?
> 2. In a two tower model what is the role of the observation tower?   

> **Answers:**
> 1.     
> 2.   

> **Exercise 6-14**: Previously, you worked with similarity search on movie attributes. When ranked, these results are a content based recommendations. Here you have worked with collaborative filtering algorithms. It is often a useful step to create a hybrid recommendation algorithm combining aspects of collaborative filtering and content based recommendation algorithms since both are based on similarity searches.
> 1. How could you extend the utility matrix you have been working with to create a representation suitable for use with a hybrid algorithm in a fairly simple manner?
> 2. Can the same extension to the utility matrix you have discussed also be used to add user context information?  

> **Answers:**
> 1.        
> 2.           

#### Copyright 2025 Stephen F. Elston. All rights reserved.  